# Lollipop Plot

A lollipop plot displays each element of a dataset as a segment and a circle. It is usually combined with the `count` stat, and is especially useful when you have several bars of the same height.

### Table of Contents

1. [Default Presentation](#default)

2. [Parameter `stat`](#stat)

  2.1. [Parameter `orientation`](#orientation)

  2.2. [With `ggmarginal()`](#ggmarginal)

3. [Parameter `fatten`](#fatten)

4. [Parameters `slope` and `intercept`](#slope_and_intercept)

  4.1. [Parameters `orientation` and `dir` (with `slope=0.5`)](#orientation_and_dir)

In [1]:
%useLatestDescriptors
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.3.1-alpha1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.3.2.0rc1.

In [3]:
import org.jetbrains.letsPlot.intern.Plot
import jetbrains.datalore.plot.base.stat.regression.LinearRegression

In [4]:
%use dataframe

In [5]:
val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv")
df.head(3)

DataFrame: rowsCount = 3, columnsCount = 12

In [6]:
val data = mapOf(
    "x" to listOf(-2, -1, 0, 1, 2),
    "y" to listOf(2, 4, 5, 1, 3)
)

<a id="default"></a>

### 1. Default Presentation

In [7]:
letsPlot(data) { x = "x"; y = "y" } + geomLollipop()

<a id="stat"></a>

### 2. Parameter `stat`

In [8]:
gggrid(
    listOf(
        letsPlot(df.toMap()) { x = "class" } + geomLollipop(stat = Stat.count()) + ggtitle("stat='count'"),
        letsPlot(df.toMap()) { x = "hwy" } + geomLollipop(stat = Stat.bin()) + ggtitle("stat='bin'"),
        letsPlot(df.toMap()) { x = "hwy" } + geomLollipop(stat = Stat.density(n=30)) + ggtitle("stat='density'")
    )
)

<a id="orientation"></a>

#### 2.1. Parameter `orientation`

In [9]:
letsPlot(df.toMap()) { y = "class" } +
    geomLollipop(stat = Stat.count(), orientation = "y") +
    themeMinimal()

<a id="ggmarginal"></a>

#### 2.2. With `ggmarginal()`

In [10]:
letsPlot(df.toMap()) { x = "hwy"; y = "cty" } +
    geomBin2D(binWidth = 1 to 1) +
    ggmarginal("r", size=.2,
               layer = geomLollipop(stat = Stat.count(), orientation = "y", size = 1) { color = "..count.." } )

<a id="fatten"></a>

### 3. Parameter `fatten`

In [11]:
gggrid(
    listOf(
        letsPlot(data) { x = "x"; y = "y"} + geomLollipop() + ggtitle("fatten=2.5 (default)"),
        letsPlot(data) { x = "x"; y = "y"} + geomLollipop(fatten = 5) + ggtitle("fatten=5"),
    )
)

<a id="slope_and_intercept"></a>

### 4. Parameters `slope` and `intercept`

In [12]:
val xs = df["hwy"].toList().map { (it as Number).toDouble() }
val ys = df["cty"].toList().map { (it as Number).toDouble() }

fun linearModel(x: Double) = LinearRegression(xs, ys, 0.95).evalX(x).y

val intercept = linearModel(0.0)
val slope = linearModel(1.0) - intercept

letsPlot(df.toMap()) { x = "hwy"; y = "cty" } +
    geomSmooth(level = 0.99) + 
    geomLollipop(slope = slope, intercept = intercept,
                 size = 1.2, shape = 21, color = "black", fill = "magenta") +
    coordFixed()

<a id="orientation_and_dir"></a>

#### 4.1. Parameters `orientation` and `dir` (with `slope=0.5`)

In [13]:
fun getPlot(orientation: String, direction: String): Plot {
    val slope = 0.5
    var p = letsPlot(data) { x = "x"; y = "y" } +
            geomABLine(slope = slope, orientation = orientation) +
            geomLollipop(slope = slope, orientation = orientation, dir = direction) +
            ggtitle("orientation='$orientation', dir='$direction'".format(orientation, direction)) +
            themeMinimal()
    if (direction == "s") {
        p += coordFixed()
    }
    return p
}

gggrid(
    listOf(
        getPlot("x", "v"), getPlot("x", "h"), getPlot("x", "s"),
        getPlot("y", "v"), getPlot("y", "h"), getPlot("y", "s"),
    ), 
    ncol = 3
)